# miRNA Pipeline

Notice: Please open the notebook under /notebooks/awsCluster/BasicCFNClusterSetup.ipynb to install CFNCluster package on your Jupyter-notebook server before running the notebook.

# 1. Configure AWS key pair, data location on S3 and the project information

In [ ]:
import os
import sys

## S3 input and output address.
s3_input_files_address = "s3://path/to/s3_input_files_address"
s3_output_files_address = "s3://path/to/s3_output_files_address"

## CFNCluster name
your_cluster_name = "cluster_name"

## The private key pair for accessing cluster.
private_key = "/path/to/aws_priate_key.pem"

## Project information
project_name = "miRNA_analysis"

## If delete cfncluster after job is done.
delete_cfncluster = False

print("Variables set.")


# 2. Create CFNCluster

Notice: The CFNCluster package can be only installed on Linux box which supports pip installation.

In [ ]:
sys.path.append("../../src/cirrus_ngs")
from cfnCluster import CFNClusterManager, ConnectionManager
## Create a new cluster
master_ip_address = CFNClusterManager.create_cfn_cluster(cluster_name=your_cluster_name)
ssh_client = ConnectionManager.connect_master(hostname=master_ip_address,
               username="ec2-user",
               private_key_file=private_key)

# 3. Run the miRNA sequencing pipeline

In [ ]:
from miRNASeq import SmallRNAPipelineManager
from util import DesignFileLoader

## "fastqc", "bowtie-alignment", "counting"
analysis_steps = ["fastqc", "trim"]
    
## load design file by calling a function load_design_file in GroupFileMaker.
design_file = "/path/to/design_file.txt"
sample_list, group_list = DesignFileLoader.load_design_file(design_file)
    
SmallRNAPipelineManager.execute(ssh_client, project_name, analysis_steps, s3_input_files_address,
                       sample_list, group_list, s3_output_files_address)